In [26]:
import pymongo
from pymongo import MongoClient, GEOSPHERE
import shapely.geometry
import pandas as pd
from pandas import DataFrame
from dotenv import load_dotenv
import os
import requests
import json
from functools import reduce
import operator
import geopandas as gpd
import cartoframes
from cartoframes.viz import Map, Layer, popup_element
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

### 1. Selecting company from MongoDB dataset.

In [4]:
client = MongoClient()
db = client.companies

### We are going to select a possible location for the office based on the available information by using Mongo db.

The applied filter assums a **number of emplooyees of 90**. 

The reason? Currently, a huge number of companies have implanted a ***work from home policy*** but, additionally the company **expect to increase their employees number in the future**, so being continously changing the location would be counterproductive.

In [7]:
query = {"number_of_employees": {"$eq":90},"offices.city":{"$ne": "",}}
project1 = {"_id":0,"offices.city": 1,"number_of_employees": 1}
results1 = db.offices.find(filt1,project1).sort("number_of_employees",1)

In [8]:
pd.DataFrame.from_records(results1)

,number_of_employees,offices
0,90,{'city': 'Amsterdam'}
1,90,{'city': 'San Francisco'}
2,90,{'city': 'Singapore'}
3,90,{'city': 'Denver'}
4,90,{'city': 'San Francisco'}
5,90,{'city': 'Palo Alto'}
6,90,{'city': 'Chicago'}
7,90,{'city': 'San Francisco'}
8,90,{'city': 'Irving'}
9,90,{'city': 'South San Francisco'}


***NYC*** is the second city worldwide and the first city in the USA in ***number of Starbucks***. For this reason, we have selected ***New York City*** as the city to establish our office.

In [20]:
query_nyc = {"number_of_employees": {"$eq":90},"offices.city": {'$eq': "New York"}}
project2 = {"_id":0,"name":1,"offices.city": 1,"number_of_employees": 1,"offices.latitude":1,"offices.longitude":1}
results2 = db.offices.find(query_nyc,project1).sort("number_of_employees",1)

In [21]:
pd.DataFrame.from_records(results2)

,name,number_of_employees,offices
0,Aleri,90,"{'city': 'New York', 'latitude': 40.7592189, '..."


#### OFFICE LOCATION

Our office is finally located where Aleri company were based on NYC. The coordenates are:

In [22]:
office_lat = 40.7592189
office_long = -73.9783534

print (office_lat,office_long)

40.7592189 -73.9783534


- **Latitud** = 40.7592189

- **Longitud** = -73.97835341

In [24]:
# Dataframe that contains the information about office location with added criteria.
office_data = [("office","40.7592189","-73.9783534","POINT (-73.9783534 40.7592189)","office location")]
df_office = pd.DataFrame(office_data, columns = ["name","latitud","longitud","geometry","criteria"]) 
df_office

,name,latitud,longitud,geometry,criteria
0,office,40.7592189,-73.9783534,POINT (-73.9783534 40.7592189),office location


## Map visualization

In [27]:
map_1 = folium.Map(location = [office_lat, office_long], zoom_start = 15)
office_loc = folium.Marker(location = [office_lat, office_long], tooltip = "Office location proposal")
icono = Icon(color = "blue",
             prefix = "fa",
             icon = "building-o",
             icon_color = "black",
             tooltip = "Office location proposal")

In [28]:
# We apply the previously defined variables in the marker function.
office_location = [office_lat, office_long]
marker_office = Marker(location = office_location, icon = icono)
marker_office.add_to(map_1)
map_1